In [33]:
import pickle
import os
os.getcwd()
os.chdir('/home/jecosine/Courses/Courses/COMP8800/Projects/CatBoostPlayground')
from allib.metrics import get_metrics
from allib.metrics.distance import AVAIL_DIST_METRICS
from allib.datasets import load_uci, AVAIL_DATASETS
from allib.models.al import ALL_STRATEGIES
import numpy as np
import pandas as pd

In [35]:
BASE_PATH = "app/comp_dist/"
SUFFIX = "100iter_catonly_wo_n"
ALSTRATEGY = "widc"

In [15]:
def get_cache_name(alstrategy, dsn, model, stg, metric, encode, suffix):
    return f"{dsn.replace('/', '_')}@{model}@{stg}_{metric}_{encode}@{suffix}@x20.pkl"

def load_dataset(dsn: str):
    ds = load_uci(dsn)
    ds.with_preprocess(
        steps=[
            "sample_n", 
            "categorical_only",
            # "continuous_to_categorical",
            "remove_constant_columns", 
            "drop_duplicate_rows"
        ],  
        params_list=[
            {"n": 1000, "random_state": 0},
            # {"encode": "ordinal"},
            {},
            {}, 
            {}
        ], 
        in_place=True
    )
    for col in ds._data.columns:
        if ds._data[col].dtype == "category":
            ds._data[col] = ds._data[col].astype("category").cat.codes.astype("int")
    ds._data.reset_index(drop=True, inplace=True)
    
    return ds

In [11]:
cache_name = get_cache_name(ALSTRATEGY, "heart-disease", "catboost", "widc", "goodall4", "ordinal", SUFFIX)

with open(os.path.join(BASE_PATH + "ppl_cache_pred", cache_name), "rb") as f:
    stats = pickle.load(f)

In [39]:
def print_info(stat):
    # check each iteration
    iters = len(stat[0]["instances"])
    for i in range(len(stat)):
        if len(stat[i]["predictions"]) != iters:
            print(f"[WARN] iteration {i} has {len(stat[i]['predictions'])} predictions ({iters})")
        # iters = len(stat[i]["instances"])
        
    return f"ntimes: {len(stat)}; niteration: {iters};"

In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

ds = load_dataset("adult")

f1_score(stats[0]["predictions"][1], ds.test_y, average="macro")

0.4478114478114478

In [54]:
import numpy as np
np.random.seed(0)
a = np.random.randint(2, 10, 100)
np.unique(a, return_counts=True)
# np.bincount(a, minlength=5)

(array([2, 3, 4, 5, 6, 7, 8, 9]), array([15, 15,  9, 16, 10,  9, 10, 16]))

In [43]:
for m in AVAIL_DIST_METRICS:
    cache_name = get_cache_name(ALSTRATEGY, "heart-disease", "catboost", "typiclust", m, "ordinal", "catonly_int")
    with open(os.path.join(BASE_PATH + "ppl_cache_pred", cache_name), "rb") as f:
        stats = pickle.load(f)
        print_info(stats)

FileNotFoundError: [Errno 2] No such file or directory: 'app/comp_dist/ppl_cache_pred/heart-disease@catboost@typiclust_euclidean_ordinal@catonly_int@x20.pkl'